In this notebook, we focus on the glasses/sunglasses category. We will exactly the same thing as for the primary classification, but instead we will try to differentiate between 2 subcategories: glasses and sunglasses.

In [ ]:
import io
import itertools

import numpy as np # for the datasets
import sklearn.metrics

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import matplotlib.pyplot as plt # for the cm

In [20]:
# Loading the datasets
data_train = np.load(r"Dataset/Glasses & Sunglasses - Train.npz") # .npz format is a numpy extension for storing multiple numpy arrays (contains 2 arrays: "labels" and "images")
data_val = np.load(r"Dataset/Glasses & Sunglasses - Validation.npz")
data_test = np.load(r"Dataset/Glasses & Sunglasses - Test.npz")

In [21]:
# Extracting the arrays from the imported data
images_train = data_train['images']
labels_train = data_train['labels']

images_val = data_val['images']
labels_val = data_val['labels']

images_test = data_test['images']
labels_test = data_test['labels']

In [22]:
# Scaling the pixel values of all images
images_train = images_train/255.0
images_val = images_val/255.0
images_test = images_test/255.0

In [23]:
# Defining constants
EPOCHS = 15 # to prevent the training from becoming too long
BATCH_SIZE = 64 # not an hyperparameter to tune (in general, the batch size may affect the speed of the training, but not the accuracy* not true for every network, dataset and problem)

In [24]:
# Defining the hyperparameters we would tune, and their values to be tested
HP_FILTER_SIZE = hp.HParam('filter_size', hp.Discrete([3,5]))
HP_FILTER_NUM = hp.HParam('filters_number', hp.Discrete([32,64]))

METRIC_ACCURACY = 'accuracy'

# Logging setup info
with tf.summary.create_file_writer(r'Logs/Model 1/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams=[HP_FILTER_SIZE, HP_FILTER_NUM],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

In [25]:
# Wrapping our model and training in a function
def train_test_model(hparams, session_num):
    
    # Outlining the model/architecture of our CNN
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(hparams[HP_FILTER_NUM], hparams[HP_FILTER_SIZE], activation='relu', input_shape=(120,90,3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(hparams[HP_FILTER_NUM], 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(2)
    ])
    
    # Defining the loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    # Compiling the model
    model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

    # Defining the logging directory
    log_dir = "Logs/Model 1/fit/" + "run-{}".format(session_num)
    
    
    def plot_confusion_matrix(cm, class_names):
        """
        Returns a matplotlib figure containing the plotted confusion matrix.

        Args:
          cm (array, shape = [n, n]): a confusion matrix of integer classes
          class_names (array, shape = [n]): String names of the integer classes
        """
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix.
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        return figure
    
    
    
    def plot_to_image(figure):
        """Converts the matplotlib plot specified by 'figure' to a PNG image and
        returns it. The supplied figure is closed and inaccessible after this call."""
        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        # Closing the figure prevents it from being displayed directly inside
        # the notebook.
        plt.close(figure)
        buf.seek(0)
        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)
        # Add the batch dimension
        image = tf.expand_dims(image, 0)
        return image
    
    
    # Defining a file writer for Confusion Matrix logging purposes
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')     
    
    
    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        test_pred_raw = model.predict(images_val)
        test_pred = np.argmax(test_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = sklearn.metrics.confusion_matrix(labels_val, test_pred)
        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=['Glasses', 'Sunglasses'])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)
    
    
    
    # Define the Tensorboard and Confusion Matrix callbacks.
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

    
    # Defining early stopping to prevent overfitting
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor = 'val_loss',
        mode = 'auto',
        min_delta = 0,
        patience = 2,
        verbose = 0, 
        restore_best_weights = True
    )
    
    # Training the model
    model.fit(
        images_train,
        labels_train,
        epochs = EPOCHS,
        batch_size = BATCH_SIZE, # if the batch_size parameter is set, TensorFlow would automatically SHUFFLE and BATCH the NumPy arrays (that is why we didn't do this earlier)
        callbacks = [tensorboard_callback, cm_callback, early_stopping],
        validation_data = (images_val,labels_val), # tuple of the numpy arrays
        verbose = 2 # verbosity to 2 = limited printable information as to not clutter the screen
    )
    
    
    # Evaluating the model's performance on the validation set
    _, accuracy = model.evaluate(images_val,labels_val) # important to make this evluation on the validation set and not on the test set, as we are yet to finalise the model
    
    # the model at that point is lost, as the variable that contains it is lost (the local variable is not logged!)
    # so if want to test the model with a different dataset or continue the training we won't be able to do it
    # hence, we will also export the model:
    # Saving the current model for future reference
    model.save(r"saved_models/Model 1/Run-{}".format(session_num)) # takes a lot of space so be careful
    
    return accuracy

In [26]:
# Creating a function to log the resuls
def run(log_dir, hparams, session_num):
    
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [27]:
session_num = 1

for filter_size in HP_FILTER_SIZE.domain.values:
    for filter_num in HP_FILTER_NUM.domain.values:

        hparams = {
            HP_FILTER_SIZE: filter_size,
            HP_FILTER_NUM: filter_num
        }

        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('Logs/Model 1/hparam_tuning/' + run_name, hparams, session_num)

        session_num += 1

--- Starting trial: run-1
{'filter_size': 3, 'filters_number': 32}
Epoch 1/15
63/63 - 35s - loss: 0.3618 - accuracy: 0.8463 - val_loss: 0.2052 - val_accuracy: 0.9440
Epoch 2/15
63/63 - 34s - loss: 0.1585 - accuracy: 0.9450 - val_loss: 0.2076 - val_accuracy: 0.9420
Epoch 3/15
63/63 - 33s - loss: 0.1354 - accuracy: 0.9590 - val_loss: 0.2069 - val_accuracy: 0.9440
16/16 [==============================] - 1s 64ms/step - loss: 0.2052 - accuracy: 0.9440
INFO:tensorflow:Assets written to: saved_models/Model 1/Run-1/assets
--- Starting trial: run-2
{'filter_size': 3, 'filters_number': 64}
Epoch 1/15
63/63 - 73s - loss: 0.4430 - accuracy: 0.8146 - val_loss: 0.2236 - val_accuracy: 0.9420
Epoch 2/15
63/63 - 75s - loss: 0.1530 - accuracy: 0.9520 - val_loss: 0.2170 - val_accuracy: 0.9600
Epoch 3/15
63/63 - 79s - loss: 0.1351 - accuracy: 0.9585 - val_loss: 0.2070 - val_accuracy: 0.9500
Epoch 4/15
63/63 - 67s - loss: 0.1368 - accuracy: 0.9588 - val_loss: 0.1960 - val_accuracy: 0.9560
Epoch 5/15
63/63

In [28]:
# Loading a model to evaluate on the test set
model = tf.keras.models.load_model(r"saved_models/Model 1/Run-1")

In [29]:
test_loss, test_accuracy = model.evaluate(images_test,labels_test)

16/16 [==============================] - 1s 74ms/step - loss: 0.1532 - accuracy: 0.9480


In [30]:
# Printing the test results
print('Test loss: {0:.4f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.1532. Test accuracy: 94.80%


In [31]:
%load_ext tensorboard
%tensorboard --logdir "Logs/Model 1/hparam_tuning"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6010 (pid 10546), started 4:47:41 ago. (Use '!kill 10546' to kill it.)

In [18]:
%load_ext tensorboard
%tensorboard --logdir "Logs/Model 1/fit"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
